#### Dataloader 

In [1]:
import sys
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import ase.io
import random

import torch
import argparse
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from torch_geometric.data import Batch
import os

from ocpmodels.datasets import TrajectoryLmdbDataset, data_list_collater
from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.common.utils import get_pbc_distances
from pymatgen.io.ase import AseAtomsAdaptor
from ase.neighborlist import NeighborList, NewPrimitiveNeighborList, PrimitiveNeighborList
import ase.db.sqlite
import ase.io.trajectory
import numpy as np
import torch
from pymatgen.io.ase import AseAtomsAdaptor
from torch_geometric.data import Data
from tqdm.notebook import tqdm
import pickle

from ocpmodels.common.utils import collate

In [27]:
dataset_config = {
    "src": "/checkpoint/electrocatalysis/relaxations/features/struct_to_energy_forces/val/is_1M"
}

dataset = TrajectoryLmdbDataset(dataset_config)

data_loader = DataLoader(
    dataset,
    batch_size=256,
    shuffle=True,
    collate_fn=data_list_collater,
    num_workers=64,
)

In [28]:
batch = next(iter(data_loader))

In [87]:
batch.y.numpy().shape

(256,)

In [31]:
natoms = batch.natoms.tolist()

In [96]:
k = torch.cumsum(batch.natoms, 0)
forces = batch.force.cpu().detach().numpy()

In [101]:
j = np.split(forces, k[:-1], axis=0)

In [108]:
len(j[-1])

124

In [76]:
j = np.split(force, np.cumsum(natoms)[:-1], 0)

In [82]:
len(j[-1])

124

In [26]:
len(j)

257

In [ ]:
def get_neighbors_ase(image, cutoff):
    n = NeighborList(cutoffs=[cutoff / 2.0] * len(image),
        self_interaction=False, skin=0, bothways=True,
        primitive=NewPrimitiveNeighborList
    )
    n.update(image)
    return [n.get_neighbors(index) for index in range(len(image))]

edge_len_ase = 0
for i in images:
    n = get_neighbors_ase(i, 6)
    for k in n:
        edge_len_ase += len(k[0])   

In [ ]:
edge_len_ase = 0
for i in images:
    n = get_neighbors_ase(i, 6)
    for k in n:
        edge_len_ase += len(k[0])   

In [ ]:
edge_len_ase

In [ ]:
print(torch.unique(edge_index, dim = 1).shape)
print(edge_index.shape)

In [ ]:
len(images)

In [ ]:
from itertools import permutations, combinations

comb = combinations(["Cu", "C", "O", "H"], 2) 

In [ ]:
for i in comb:
    print(i)

In [ ]:
test = ase.io.trajectory.Trajectory("/checkpoint/electrocatalysis/relaxations/bulkadsorbate/chunk27/random837409.traj")[2]

In [ ]:
test.get_forces(apply_constraint=False).mean()

In [ ]:
data = next(iter(data_loader))
pos = data.pos

In [ ]:
data.pos.shape[0]

In [ ]:
torch.arange(len(data.edge_index[1]))[(data.edge_index[1] == 0)]

In [ ]:
data.pos.shape[0]

In [ ]:
out = get_pbc_distances(data.pos, data.edge_index, data.cell, data.cell_offsets, data.neighbors, return_offsets=True, max_neigh=10)

In [ ]:
edge_index = out["edge_index"]

In [ ]:
distances = out["distances"]

In [ ]:
distances[:10]

In [ ]:
radius_index.shape

In [ ]:
max_distance = []
for data in tqdm(data_loader):
    edge_index, edge_weight = get_pbc_distances_fix(data.pos, data.edge_index, data.cell, data.cell_offsets, data.nghbrs)
    max_distance.append(edge_weight.max().item())

In [ ]:
sum(torch.tensor(max_distance)>6)

In [ ]:
edge_index, edge_weight = get_pbc_distances(data.pos, data.edge_index, data.cell, data.cell_offsets, data.natoms)

In [ ]:
edge_weight.max().item()

In [ ]:
data = dataset[0]
edge_index, edge_weight = get_pbc_distances(
    data.pos, 
    data.edge_index, 
    data.cell, 
    data.cell_offsets, 
    data.natoms
)
print(edge_weight.max())

In [ ]:
path = "/private/home/mshuaibi/baselines/expts/embeddings/"
atom_map = torch.stack(torch.load(os.path.join(path, "cgcnn_embeddings.pt")))

In [ ]:
z = data.atomic_numbers.long()
pos = data.pos
row, col = edge_index
edge_weight = pos[row] - pos[col]
edge_weight = edge_weight.norm(dim=-1)
edge_weight.shape
pm2ang = 100
bond_criteria = torch.abs(edge_weight-(atom_map[z[row]][:, 3] + atom_map[z[col]][:, 3])/pm2ang)
bonded_edge = (bond_criteria <= 0.5).nonzero().view(-1)
vdw_edge = (bond_criteria > 0.5).nonzero().view(-1)

In [ ]:
edge_index.shape

In [ ]:
vdw_edge.shape

In [ ]:
bond_edge_index = data.edge_index[:, bonded_edge]
vdw_edge_index = data.edge_index[:, vdw_edge]

In [ ]:
bond_edge_index.shape

In [ ]:
vdw_edge_index.shape

In [ ]:
edge_idx = data.edge_index
row, col = edge_idx
pos = data.pos

edge_weight = pos[row] - pos[col]
edge_weight = edge_weight.norm(dim=-1)
edge_weight

In [ ]:
edge_index = data.edge_index
row, col = edge_index

edge_weight_pbc = pos[row] - pos[col]
# correct for pbc
cell = torch.repeat_interleave(data.cell, data.natoms * 12, dim=0)
cell_offsets = data.cell_offsets
offsets = (
    cell_offsets.float()
    .view(-1, 1, 3)
    .bmm(cell.float())
    .view(-1, 3)
)
edge_weight_pbc = -pos[row] + pos[col] + offsets
# edge_weight_pbc += offsets
# compute distances
edge_weight_pbc = edge_weight_pbc.norm(dim=-1)

In [ ]:
data_loader = DataLoader(
    dataset,
    batch_size=256,
    shuffle=False,
    collate_fn=data_list_collater,
    num_workers=64,
)

relax = []
for batch in data_loader:
    free_atom_idx = batch.fixed == 0
    relax.append(batch.relax_vector[free_atom_idx])

In [ ]:
relax = torch.cat(relax, 0)

In [ ]:
mean_vector = torch.tensor([f.item() for f in relax.mean(0)])
mean_vector

In [ ]:
torch.mean(torch.abs(relax-mean_vector), dim=0)

In [ ]:
energies = torch.cat(energies, 0).view(-1, 1)

In [ ]:
pos = data.pos
edge_index = data.edge_index
row, col = edge_index
edge_weight = pos[row] - pos[col]

cell = torch.repeat_interleave(data.cell, data.natoms * 12, dim=0)
cell_offsets=data.cell_offsets
offsets = (
cell_offsets.float().view(-1,1,3).bmm(cell.float()).view(-1, 3))
edge_weight += offsets

In [ ]:
bulk_mask = (data.tags == 0).nonzero()
surface_mask = (data.tags == 1).nonzero()
ads_mask = (data.tags == 2).nonzero()

In [ ]:
tag_coeff = [1, 1, 1]
bulk_loss = tag_coeff[0] * torch.mean(data.force[bulk_mask]) * 3 *bulk_mask.size(0)
surface_loss = tag_coeff[2] * torch.mean(data.force[surface_mask]) * 3 *surface_mask.size(0)
ads_loss = tag_coeff[2] * torch.mean(data.force[ads_mask]) * 3 *ads_mask.size(0)
weighted_force_loss = (bulk_loss + surface_loss + ads_loss)/(3*data.tags.size(0))

In [ ]:
energies, forces = [], []

for i, batch in tqdm(enumerate(data_loader)):
    energies.append(batch.y)
#     free_atom_idx = batch.fixed ==0
#     forces.append(batch.force[free_atom_idx])
    forces.append(batch.force)

In [16]:
images = ase.io.read("/checkpoint/sidgoyal/electro_done/random1255319.traj", ":")
energies = [image.get_potential_energy() for image in images]

In [22]:
images[0].get_calculator().parameters

{'kpoints_generation': {'divisions': array([3, 3, 1]),
  'usershift': array([0., 0., 0.]),
  'genvec1': array([0.33333333, 0.        , 0.        ]),
  'genvec2': array([0.        , 0.33333333, 0.        ]),
  'genvec3': array([ 0., -0.,  1.]),
  'shift': array([0., 0., 0.])},
 'system': 'unknown system',
 'lcompat': False,
 'prec': 'normal',
 'enmax': 350.0,
 'enaug': 644.873,
 'ediff': 0.0001,
 'ialgo': 38,
 'iwavpr': 11,
 'nbands': 486,
 'nelect': 805.0,
 'turbo': 0,
 'irestart': 0,
 'nreboot': 0,
 'nmin': 0,
 'eref': 0.0,
 'ismear': 1,
 'sigma': 0.2,
 'kspacing': 0.5,
 'kgamma': True,
 'lreal': True,
 'ropt': array([-0.0005, -0.0005, -0.0005, -0.0005, -0.0005]),
 'lmaxpaw': -100,
 'lmaxmix': 2,
 'nlspline': False,
 'istart': 0,
 'icharg': 2,
 'iniwav': 1,
 'ispin': 1,
 'lnoncollinear': False,
 'magmom': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.,

In [ ]:
idx = 0
length = len(images[0])
pos_vectors = torch.zeros(5, length, 3)
for j, i in enumerate(range(idx, idx-10, -1)):
    try:
        assert i >= 1
        current_pos = images[i].positions
        previous_pos = images[i-1].positions
        pos_vectors[j] = current_pos - previous_pos
    except:
        break

In [ ]:
plt.plot(range(len(energies)), energies)
plt.xlabel("step")
plt.ylabel("energy")
plt.title("random1255319")

In [ ]:
import ase.io
import os
from matplotlib import pyplot as plt
import pickle
from tqdm.notebook import tqdm
import multiprocessing as mp
from ase.visualize import view

In [ ]:
k = open("/private/home/mshuaibi/ocp-modeling/utils/val_sample.txt", "r")
traj_paths = k.read().splitlines()
k = open("/checkpoint/mshuaibi/mappings/adslab_ref_energies_ocp728k.pkl", "rb")
ref = pickle.load(k)
ml_dir = "/private/home/mshuaibi/baselines/expts/relaxations/ml_results/"

In [ ]:
j = open("/checkpoint/mshuaibi/mappings/sysid_to_bulkads_dir.pkl", "rb")
bulkads_dir = pickle.load(j)
randomids = list(bulkads_dir)

In [ ]:
k = open("/checkpoint/mshuaibi/mappings/sysid_to_metadata.pkl", "wb")

In [ ]:
def get_meta(randomid):
    try:
        path = bulkads_dir[randomid]
        t = open(os.path.join(path, "metadata.pkl"), "rb")
        metadata = pickle.load(t)
        return (randomid, metadata["adsorbed_bulk_metadata"])
    except:
        pass

In [ ]:
metadata_mappings = {}

In [ ]:
pool = mp.Pool(80)
metadata = list(tqdm(pool.imap(get_meta, randomids), total=len(randomids)))

In [ ]:
for i in metadata:
    try:
        metadata_mappings[i[0]] = i[1]
    except:
        continue

In [ ]:
pickle.dump(metadata_mappings, k)

In [ ]:
# for i in traj_paths:
i = traj_paths[6]
print(i)
randomid = os.path.basename(i)[:-5]
dft_images = ase.io.read(i, ":")
dft_energies = [image.get_potential_energy(apply_constraint=False) for image in dft_images]

systemdir = os.path.join(ml_dir, randomid)
ml_images = ase.io.read(systemdir+f"/ml_idx_0_{randomid}.traj", ":")
ml_energies = [image.get_potential_energy(apply_constraint=False) + ref[randomid] for image in ml_images]

plt.plot(range(len(dft_energies)), dft_energies, label="DFT")
plt.plot(range(len(ml_energies)), ml_energies, label="ML")
plt.xlabel("Relaxation step")
plt.ylabel("Energy, eV")
plt.legend()

In [ ]:
dft_relax = dft_images[-1]
fixed = dft_relax.constraints[0].index
idxs = range(len(dft_relax))
free = [i for i in idxs if i not in fixed]
dft_relaxed_pos = dft_relax.positions[free]
ml_relaxed_pos = ml_images[-1].positions[free]

np.mean(np.abs(dft_relaxed_pos - ml_relaxed_pos))

In [ ]:
np.mean(np.abs(dft_images[0].positions[free]-dft_relaxed_pos))

In [ ]:
k = open("/checkpoint/mshuaibi/mappings/sysid_to_bulk_adsorbate_idx.pkl", "rb")
admaps = pickle.load(k)
j = open("/checkpoint/mshuaibi/ocpdata_reset_07_13_20/ocpdata_train_CO.txt", "a")
n = open("/checkpoint/mshuaibi/ocpdata_reset_07_13_20/ocpdata_val_CO.txt", "a")
l = open("/checkpoint/mshuaibi/ocpdata_reset_07_13_20/ocpdata_train628k_raw.txt", "r")
m = open("/checkpoint/mshuaibi/ocpdata_reset_07_13_20/ocpdata_val50k_raw.txt", "r")
train_paths = l.read().splitlines()
val_paths = m.read().splitlines()

In [ ]:
count = 0
co_paths = []
oh_paths = []
nh_paths = []
n_large_paths = []
large_ads_paths = []
single_alloy_paths = []
ternary_paths = []
for path in tqdm(train_paths):
    try:
        randomid = os.path.basename(path)[:-5]
        if admaps[randomid]["ads"] == 5:
            co_paths.append(path)
        elif admaps[randomid]["ads"] == 2:
            oh_paths.append(path)
        elif admaps[randomid]["ads"] == 72:
            nh_paths.append(path)
        elif admaps[randomid]["ads"] == 58:
            n_large_paths.append(path)
        elif admaps[randomid]["ads"] == 56:
            large_ads_paths.append(path)
        elif admaps[randomid]["bulk"] == 22:
            single_alloy_paths.append(path)
        elif admaps[randomid]["bulk"] == 27:
            single_alloy_paths.append(path)
        elif admaps[randomid]["bulk"] < 100:
            single_alloy_paths.append(path)
        elif admaps[randomid]["bulk"] >11470:
            ternary_paths.append(path)
    except:
        continue

In [ ]:
os.makedirs("traj4landing_page", exist_ok=True)

In [ ]:
idx = random.sample(ternary_paths, 1)[0]

randomid = os.path.basename(idx)
images = ase.io.read(idx, ":")
print(len(images))
ase.io.write("./traj4landing_page/ternary.png", images[0], rotation = "-75x, 45y, 10z")

#### Filter insights

In [ ]:
import ase.io
from ocpmodels.preprocessing import AtomsToGraphs
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
import seaborn as sns
from tqdm.notebook import tqdm
import torch
import numpy as np

from ocpmodels.datasets import TrajectoryLmdbDataset, data_list_collater

In [ ]:
dataset_config = {
    "src": "/checkpoint/electrocatalysis/relaxations/features/struct_to_energy_forces/train/200k"
}

import time
t = time.time()
dataset = TrajectoryLmdbDatasetprofile(dataset_config)
print(time.time()-t)

In [ ]:
len(dataset)

In [ ]:
dataset = dataset[:1000]

In [ ]:
data_loader = DataLoader(
    dataset,
    batch_size=512,
    shuffle=True,
    collate_fn=data_list_collater,
    num_workers=64,
)

In [ ]:
energies = []
forces = []

i = 0
for batch in tqdm(data_loader):
    energies.append(batch.y)
    forces.append(batch.force)
    i += 1
    if i >= 10000:
        break

In [ ]:
energies = energies.numpy()

In [ ]:
energies.mean()

In [ ]:
energies.stdev()

In [ ]:
forces = torch.cat(forces,0)

In [ ]:
forces.shape

In [ ]:
import seaborn as sns

sns.distplot(energies)

In [ ]:
max_displacement = []
mean_displacement = []

In [ ]:
count = 0
for data in tqdm(dataset):
    free = (data.tags != 0).nonzero().flatten()
    mean_displacement.append(data.relax_vector[free].abs().norm(dim=-1).mean().item())
    max_displacement.append(data.relax_vector[free].abs().norm(dim=-1).max().item())
    count += 1
    if count >= 1000000:
        break

In [ ]:
plt.figure(figsize=(8,8))
sns.distplot(max_displacement)
plt.xlabel("Max atom displacement from initial, $\AA$", fontsize=25)
plt.ylabel("Distribution", fontsize=25)
plt.tick_params(labelsize=25)
plt.show()

In [ ]:
max_displacement = np.array(max_displacement)
mean_displacement = np.array(mean_displacement)

In [ ]:
(max_displacement>40).nonzero()

In [ ]:
ase.io.write("test1.traj", ase.io.read(traj_paths[24413], ":"))

In [ ]:
t= ase.io.read(traj_paths[53365], "0")

In [ ]:
t.cell

In [ ]:
t = ase.io.read(traj_paths[24413], "32")

t.get_masses()

In [ ]:
t.get_atomic_numbers()

In [ ]:
ase.io.write("test2.traj", ase.io.read(traj_paths[53350], ":"))

In [ ]:
max_tol = [5, 10, 15, 20, 30, 50]

for i in max_tol:
    count = (max_displacement>i).nonzero()[0]
    p = len(count)/len(max_displacement)
    print(i, p)

In [ ]:
plt.figure(figsize=(8,8))
sns.distplot(mean_displacement)
plt.xlabel("Average atom displacement from initial, $\AA$", fontsize=25)
plt.ylabel("Distribution", fontsize=25)
plt.tick_params(labelsize=25)
plt.show

In [1]:
import argparse
import multiprocessing as mp
import os
import pickle
import random

from tqdm import tqdm

import ase.io
import lmdb
import torch
from ase import Atoms
from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.datasets import data_list_collater, SinglePointLmdbDataset
import copy

In [2]:
db_path = "/private/home/mshuaibi/baselines/ocpmodels/common/efficient_validation/water_pair/data_relax.lmdb"
images = ase.io.read("/private/home/mshuaibi/baselines/ocpmodels/common/efficient_validation/water_relax.traj", ":")

In [10]:
db = lmdb.open(
    db_path,
    map_size=1099511627776 * 2,
    subdir=False,
    meminit=False,
    map_async=True,
)

In [4]:
a2g = AtomsToGraphs(
    max_neigh=50,
    radius=6,
    r_energy=True,
    r_forces=True,
    r_distances=False,
    r_fixed=True,
)
dl = a2g.convert_all(images, disable_tqdm=True)

In [5]:
d = dl[0]
d

Data(atomic_numbers=[3], cell=[1, 3, 3], cell_offsets=[6, 3], edge_index=[2, 6], fixed=[3], force=[3, 3], natoms=3, pos=[3, 3], y=2.7696319984994133)

In [6]:
d.pos_relaxed = dl[-1].pos
d.y_relaxed = dl[-1].y
j = copy.deepcopy(d)
j.test = 3434

In [11]:
for i, j in enumerate(range(2)):
    txn = db.begin(write=True)
    txn.put(
        f"{i}".encode("ascii"), pickle.dumps(d, protocol=-1)
    )
    txn.commit()
db.sync()
db.close()

In [13]:
data1 = SinglePointLmdbDataset(dict(src="/private/home/mshuaibi/baselines/ocpmodels/common/efficient_validation/water_pair/data_relax.lmdb"))

In [15]:
len(data1)

2

In [ ]:
for do in dl:
    txn = db.begin(write=True)
    txn.put(
        f"{0}".encode("ascii"), pickle.dumps(do, protocol=-1)
    )
    txn.commit()
db.sync()
db.close()

In [ ]:
paths = [
"/checkpoint/electrocatalysis/relaxations/bulkadsorbate/chunk15/random225049.traj",
"/checkpoint/electrocatalysis/relaxations/bulkadsorbate/chunk16/random388359.traj",
"/checkpoint/electrocatalysis/relaxations/bulkadsorbate/chunk6/random139141.traj",
"/checkpoint/electrocatalysis/relaxations/bulkadsorbate/chunk18/random496361.traj",
"/checkpoint/electrocatalysis/relaxations/bulkadsorbate/chunk16/random380929.traj",
"/checkpoint/electrocatalysis/relaxations/bulkadsorbate/chunk15/random211416.traj",
"/checkpoint/electrocatalysis/relaxations/bulkadsorbate/chunk62/random2032440.traj",
"/checkpoint/electrocatalysis/relaxations/bulkadsorbate/chunk8/random1470615.traj",
"/checkpoint/electrocatalysis/relaxations/bulkadsorbate/chunk41/random1877054.traj",
"/checkpoint/electrocatalysis/relaxations/bulkadsorbate/chunk15/random348419.traj",
"/checkpoint/electrocatalysis/relaxations/bulkadsorbate/chunk4/random1336791.traj",
]

In [ ]:
import os

In [ ]:
for path in paths:
    images = ase.io.read(path, ":")
    randomid = os.path.basename(path)
    ase.io.write(randomid, images)

In [ ]:
images[0].constraints[0].index

In [ ]:
surface_elements_to_remove = [1, 3, 11, 19, 37, 55, 87, 2, 10, 18, 36, 54, 86, 7, 8, 9, 15, 16, 17, 34, 35, 53]

In [ ]:
ase.io.write("test.traj", images)

In [ ]:
k = torch.tensor([0, 0, 4, 5, 6, 7 ,8])

In [ ]:
k.nonzero().flatten()

In [ ]:
k[k != 0].flatten()

In [ ]:
en_stdev = []
f_stdev = []
size = [200000, 300000, 400000, 600000, 800000]
for i in size:
    k = open(f"eval_en_{i}.pkl", "rb")
    j = open(f"eval_fo_{i}.pkl", "rb")
    
    en = pickle.load(k)
    e_mae = en[str(i)]
    en_stdev.append(np.std(e_mae))
    
    f = pickle.load(j)
    f_mae = f[str(i)]
    f_stdev.append(np.std(f_mae))

In [ ]:
plt.plot(size, en_stdev)
plt.title("Energy")
plt.xlabel("Validation size")
plt.ylabel("Stdev")

In [ ]:
plt.plot(size, f_stdev)
plt.title("Force avg")
plt.xlabel("Validation size")
plt.ylabel("Stdev")